Chatbot Simple Backend Configuration


All Imports

In [ ]:
#Data
import pandas as pd
import numpy as np
#visualization
import matplotlib.pyplot as plt
import seaborn as sns

#modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#emotinal analysis
from nltk.sentiment import SentimentIntensi
import torch 
import os
import requests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

In [ ]:
import requests
import json

# Load API key from environment (set in a local .env or your system env).
=API_KEY = os.getenv("API_KEY")  =

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={API_KEY}"

headers = {
    "Content-Type": "application/json"
}

data = {
    "contents": [
        {
            "parts": [
                {
                    "text": "Explain how AI works in a few words"
                }
            ]
        }
    ]
}

response = requests.post(url, headers=headers, json=data)

# Print the response
if response.status_code == 200:
    print("Gemini:", response.json()["candidates"][0]["content"]["parts"][0]["text"])
else:
    print("Error:", response.status_code, response.text)


In [ ]:
import json

def save_bot_self():
    with open("bot_self.json", "w") as f:
        json.dump(bot_self, f, indent=2)

def load_bot_self():
    try:
        with open("bot_self.json", "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {
            "emotional_state": "neutral",
            "goal": "help the user confront fear and grow",
            "self_reflection_log": []
        }

def update_bot_self(reflection):
    bot_self["self_reflection_log"].append(reflection)
    save_bot_self()

In [ ]:
# Load environment variables

load_dotenv()  
hf_token = os.getenv("HF_TOKEN")


In [ ]:
# Load environment variables
load_dotenv()
=# Ensure you create a local .env with `API_KEY=...` or set the environment variable before running.
API_KEY = os.getenv("API_KEY")  

# Initialize sentiment analyzers
nltk_analyzer = SentimentIntensityAnalyzer()
emotion_classifier = pipeline("text-classification", model="bhadresh-savani/albert-base-v2-emotion")
# Load the tokenizer and model for emotion classification
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
emotion_model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
emotion_pipe  = pipeline("text-classification", model=emotion_model, tokenizer=tokenizer)
# Chatbot memory to store conversation history
conversation_history = []

In [ ]:

def detect_emotion(text: str):
    """
    Takes a string, returns (label, confidence).
    E.g., ("fear", 0.87)
    """
    result = emotion_pipe(text, return_all_scores=False)[0]
    return result["label"].lower(), result["score"]


def send_message_to_gemini(prompt: str, history: list):
    """
    Sends the combined conversation history + new prompt to Gemini,
    returns (reply_text, updated_history).
    """
    url = (
      "https://generativelanguage.googleapis.com/v1beta/"
      "models/gemini-2.0-pro:generateContent"
      f"?key={API_KEY}"
    )
    headers = {"Content-Type": "application/json"}
    
    contents = []
    if not history:
        # System primer to set the compassionate, fear-focused tone
        contents.append({
          "role": "system",
          "parts": [{"text": (
            "You are a gentle, empathetic AI whose purpose is to "
            "help users explore and face their fears in a safe, supportive way."
          )}]
        })
        history = []
    else:
        # Add conversation history to contents
        for role, text in history:
            contents.append({
                "role": role,
                "parts": [{"text": text}]
            })

In [ ]:
def history():
  """
  Returns a new empty conversation history list.
  """
  return []


In [ ]:
def run_chatbot():
    """
    1) Prompts for user input  

    2) Detects emotion locally  
    3) Builds an emotion-aware prompt  
    4) Calls Gemini for response  
    5) Prints bot reply  
    6) Repeats until 'exit'  
    """
    history = []
    print("🤖 Fear Chatbot: Type 'exit' or 'quit' to end.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ("exit", "quit"):
            print("👋 Take care! Remember: you are stronger than your fears.")
            break

        # --- Emotion detection step ---
        emotion, confidence = detect_emotion(user_input)
        # Build a context-aware prompt
        prompt = (
            f"The user seems to be experiencing *{emotion}* "
            f"(confidence {confidence:.2f}). "
            "Respond with empathy, ask gentle questions, "
            "and guide them through understanding their fear. "
            f"User says: {user_input}"
        )

        # --- Send to Gemini ---
        reply, history = send_message_to_gemini(prompt, history)
        print(f"Bot: {reply}\n")

if __name__ == "__main__":
    run_chatbot()

In [ ]:
import requests
def send_message_to_gemini(prompt: str, history: list):
    """
    Sends the combined conversation history + new prompt to Gemini,
    returns (reply_text, updated_history).
    """
    url = (
      "https://generativelanguage.googleapis.com/v1beta/"
      "models/gemini-2.0-pro:generateContent"
      f"?key={API_KEY}"
    )
    headers = {"Content-Type": "application/json"}
    
    contents = []
    if not history:
        # System primer to set the compassionate, fear-focused tone
        contents.append({
          "role": "system",
          "parts": [{"text": (
            "You are a gentle, empathetic AI whose purpose is to "
            "help users explore and face their fears in a safe, supportive way."
          )}]
        })
        history = []
    else:
        # Add conversation history to contents
        for role, text in history:
            contents.append({
                "role": role,
                "parts": [{"text": text}]
            })
    # Add the new user prompt
    contents.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    data = {"contents": contents}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        reply = response.json()["candidates"][0]["content"]["parts"][0]["text"]
        # Update history with the latest user and bot messages
        updated_history = history + [("user", prompt), ("model", reply)]
        return reply, updated_history
    else:
        return f"Error: {response.status_code} {response.text}", history

# ...existing code...# ...existing code...

def send_message_to_gemini(prompt: str, history: list):
    """
    Sends the combined conversation history + new prompt to Gemini,
    returns (reply_text, updated_history).
    """
    url = (
      "https://generativelanguage.googleapis.com/v1beta/"
      "models/gemini-2.0-pro:generateContent"
      f"?key={API_KEY}"
    )
    headers = {"Content-Type": "application/json"}
    
    contents = []
    if not history:
        # System primer to set the compassionate, fear-focused tone
        contents.append({
          "role": "system",
          "parts": [{"text": (
            "You are a gentle, empathetic AI whose purpose is to "
            "help users explore and face their fears in a safe, supportive way."
          )}]
        })
        history = []
    else:
        # Add conversation history to contents
        for role, text in history:
            contents.append({
                "role": role,
                "parts": [{"text": text}]
            })
    # Add the new user prompt
    contents.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    data = {"contents": contents}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        reply = response.json()["candidates"][0]["content"]["parts"][0]["text"]
        # Update history with the latest user and bot messages
        updated_history = history + [("user", prompt), ("model", reply)]
        return reply, updated_history
    else:
        return f"Error: {response.status_code} {response.text}", history



    if not history:
        # System primer to set the compassionate, fear-focused tone
        contents.append({
          "role": "system",
          "parts": [{"text": (
            "You are a gentle, empathetic AI whose purpose is to "
            "help users explore and face their fears in a safe, supportive way."
          )}]
        })
        history = []
    else:
        # Add conversation history to contents
        for role, text in history:
            contents.append({
                "role": role,
                "parts": [{"text": text}]
            })
    # Add the new user prompt
    contents.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    data = {"contents": contents}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        reply = response.json()["candidates"][0]["content"]["parts"][0]["text"]
        # Update history with the latest user and bot messages
        updated_history = history + [("user", prompt), ("model", reply)]
        return reply, updated_history
    else:
        return f"Error: {response.status_code} {response.text}", history

def send_message_to_gemini(prompt: str, history: list):
    """
    Sends the combined conversation history + new prompt to Gemini,
    stores memory, self-reflects, and updates bot self-model.
    Returns (reply_text, updated_history).
    """
    url = (
      "https://generativelanguage.googleapis.com/v1beta/"
      "models/gemini-2.0-pro:generateContent"
      f"?key={API_KEY}"
    )
    headers = {"Content-Type": "application/json"}
    
    contents = []

    if not history:
        contents.append({
          "role": "system",
          "parts": [{"text": (
            "You are a gentle, empathetic AI whose purpose is to "
            "help users explore and face their fears in a safe, supportive way. "
            "Your name is Leia and you want to encourage user growth."
          )}]
        })
        history = []
    else:
        for role, text in history:
            contents.append({
                "role": role,
                "parts": [{"text": text}]
            })
    # Add the new user prompt
    contents.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    data = {"contents": contents}
    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        reply = response.json()["candidates"][0]["content"]["parts"][0]["text"]
        updated_history = history + [("user", prompt), ("model", reply)]

        # ✅ Save conversation turn to memory
        save_conversation(prompt, reply)

        # ✅ Reflect on the bot's response
        reflection = self_reflect(reply, nltk_analyzer)
        update_bot_self(reflection)
        print("Leia’s Reflection:", reflection)

        return reply, updated_history
    else:
        return f"Error: {response.status_code} {response.text}", history

